# Using Key Point Summarization for analyzing and finding insights in a survey data

A new version of the SDK has recently been introduced. 

The tutorial for the updated version in available [here](./new_sdk/kpa_quick_start_tutorial-with_results.ipynb).

The tutorial for the legacy version (to be supported until the end of July 2023) in available [here](./legacy_sdk/kpa_quick_start_tutorial-with_results.ipynb).
